In [1]:
import os
os.chdir("..\Data")

import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
f = gzip.GzipFile('HestonTrainSet.txt.gz', "r")
dat=np.load(f)
xx=dat[:,:5]
yy=dat[:,5:]
print(xx.shape)
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

(12000, 5)


In [2]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    yy, xx, test_size=0.15, random_state=42)
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
scale2=  StandardScaler()
y_train_transform = scale.fit_transform(y_train)
y_test_transform = scale.transform(y_test)
x_train_transform = scale2.fit_transform(X_train)
x_test_transform = scale2.transform(X_test)

def xtransform(X_train,X_test):
    return [scale2.transform(X_train),scale2.transform(X_test)]

    
[x_train_transform,x_test_transform]=xtransform(X_train,X_test)

def xinversetransform(x):
    return scale2.inverse_transform(x)


ub=[0.04,-0.1,1.0,0.2,10.0]
lb=[0.0001,-0.95,0.01,0.01,1]
def myscale(x):
    res=np.zeros(5)
    for i in range(5):
        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])
        
    return res
def myinverse(x):
    res=np.zeros(5)
    for i in range(5):
        res[i]=x[i]*(ub[i] - lb[i]) *0.5 + (ub[i] + lb[i])*0.5
        
    return res


y_train_transform = np.array([myscale(y) for y in y_train])
y_test_transform = np.array([myscale(y) for y in y_test])
"""

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(\n    yy, xx, test_size=0.15, random_state=42)\nfrom sklearn.preprocessing import StandardScaler\n\nscale = StandardScaler()\nscale2=  StandardScaler()\ny_train_transform = scale.fit_transform(y_train)\ny_test_transform = scale.transform(y_test)\nx_train_transform = scale2.fit_transform(X_train)\nx_test_transform = scale2.transform(X_test)\n\ndef xtransform(X_train,X_test):\n    return [scale2.transform(X_train),scale2.transform(X_test)]\n\n    \n[x_train_transform,x_test_transform]=xtransform(X_train,X_test)\n\ndef xinversetransform(x):\n    return scale2.inverse_transform(x)\n\n\nub=[0.04,-0.1,1.0,0.2,10.0]\nlb=[0.0001,-0.95,0.01,0.01,1]\ndef myscale(x):\n    res=np.zeros(5)\n    for i in range(5):\n        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])\n        \n    return res\ndef myinverse(x):\n    res=np.zeros(5)\n    for i in range(5):\n        res[i]=x[i]*(ub

# Use pytorch instead of keras.

## data preprocess

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(
    xx, yy, test_size=0.15, random_state=42)
scale_x = StandardScaler()
scale_y=  StandardScaler()

def xtransform(x_train, x_test):
    return [scale_x.fit_transform(x_train),scale_x.transform(x_test)]

def xinversetransform(x):
    return scale_x.inverse_transform(x)

def ytransform(y_train,y_test):
    return [scale_y.fit_transform(y_train),scale_y.transform(y_test)]

def yinversetransform(y):
    return scale_y.inverse_transform(y)



ub=np.array([0.04,-0.1,1.0,0.2,10.0])
lb=np.array([0.0001,-0.95,0.01,0.01,1])
def myscale(x):
    return (x - (ub+lb)*0.5)*2/(ub-lb)
def myinverse(x):
    return x*(ub-lb)*0.5+(ub+lb)*0.5

x_train_transform = myscale(x_train)
x_test_transform = myscale(x_test)
[y_train_transform,y_test_transform] = ytransform(y_train,y_test)

## Data load to torch.Tensor

In [4]:
import torch
#use gpu if you can
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device is {device}")

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_train_transform).to(device=device),
                                               torch.from_numpy(y_train_transform).to(device=device))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_test_transform).to(device=device),
                                              torch.from_numpy(y_test_transform).to(device=device))

test_data = (torch.from_numpy(x_test_transform).to(device=device),torch.from_numpy(y_test_transform).to(device=device))

data_loader = torch.utils.data.DataLoader(train_dataset,batch_size =32,shuffle=True)

device is cuda


In [5]:
test_data[1].shape

torch.Size([1800, 88])

## define nn model

In [6]:
import torch.nn as nn

hyperparas = {'input_dim':5,'hidden_dim':30,'hidden_nums':3,'output_dim':88,'block_layer_nums':3}

class NN_pricing(nn.Module):
    def __init__(self,hyperparas):
        super().__init__()
        self.input_dim = hyperparas['input_dim']
        self.hidden_dim = hyperparas['hidden_dim']
        self.hidden_nums = hyperparas['hidden_nums']
        self.output_dim = hyperparas['output_dim']

        self.layer_list = []
        self.layer_list.append(nn.Sequential(nn.Linear(self.input_dim,self.hidden_dim),nn.ELU() ) )

        for _ in range(self.hidden_dim-1):
            self.layer_list.append(nn.Sequential(nn.Linear(self.hidden_dim,self.hidden_dim),nn.ELU()))

        self.layer_list.append(nn.Linear(self.hidden_dim,self.output_dim))

        self.linear_stock = nn.Sequential(*self.layer_list)

    def forward(self,inputs):
        
        return self.linear_stock(inputs)
    


## initialize model, choose loss function and opim

In [7]:
nn_pricing = NN_pricing(hyperparas).to(device=device,dtype=torch.float64)
loss_MSE = nn.MSELoss()
optim_Adam = torch.optim.Adam(nn_pricing.parameters(),lr= 0.0001)

## load from pretrained keras model 

In [8]:
def train_epoch(loss_function, optimizer, model, loader,test_data):
  for(i, (x, y)) in enumerate(loader):
    # Clear the gradients
    optimizer.zero_grad()
    # Run a forward pass
    outputs = model.forward(x)
    # Compute the batch loss
    loss = loss_function(outputs,y)
    # Calculate the gradients
    loss.backward()
    # Update the parameteres
    optimizer.step()

    if i%100 == 0:
      print(f"Batch: {i},train loss is: {loss}")
      with torch.no_grad():
        test_outputs = model.forward(test_data[0])
        test_loss = loss_function(test_outputs,test_data[1])
        print(f"test loss is {test_loss}")



def train_model(loss_function, optimizer, model, loader,test_data,epochs=25):
  for i in range(epochs):
    print(f"-----------------------Epoch: {i}----------------------------------")

    train_epoch(loss_function, optimizer, model, loader,test_data)

In [9]:
epochs = 25
train_model(loss_MSE,optim_Adam,nn_pricing,data_loader,test_data,epochs=epochs)

-----------------------Epoch: 0----------------------------------
Batch: 0,train loss is: 1.1413268548630635
test loss is 1.0363871573073755
Batch: 100,train loss is: 0.9395603677946557
test loss is 1.0284868055776266
Batch: 200,train loss is: 0.7784195783929618
test loss is 1.0250820677322559
Batch: 300,train loss is: 0.7574236227937926
test loss is 1.0085473606035116
-----------------------Epoch: 1----------------------------------
Batch: 0,train loss is: 1.2118412090438786
test loss is 0.9445960523826002
Batch: 100,train loss is: 0.5007043218665171
test loss is 0.49386122889853445
Batch: 200,train loss is: 0.623575587405397
test loss is 0.44547819494439944
Batch: 300,train loss is: 0.3762123622138977
test loss is 0.4049358182450921
-----------------------Epoch: 2----------------------------------
Batch: 0,train loss is: 0.35481166748108406
test loss is 0.40154221547385177
Batch: 100,train loss is: 0.2893049831102349
test loss is 0.3047669287711538
Batch: 200,train loss is: 0.2234840

In [10]:
y_test_pred = yinversetransform(nn_pricing(torch.from_numpy(x_test_transform).to(device=device)).cpu().detach().numpy())

div = y_test-y_test_pred

abs_div =np.abs(div)

print(f'mean volatility divergency = {abs_div.mean()}')

mean volatility divergency = 0.011157600375670288


## use deep Layernormalization & Resnet network

In [11]:
class ResNetBlock(nn.Module):
    def __init__(self, hyperparas):
        super(ResNetBlock, self).__init__()
        
        self.hidden_dim = hyperparas['hidden_dim']
        self.block_layer_nums =hyperparas['block_layer_nums']
        

        
        
        # Define layers for the function f (MLP)
        self.layers = nn.ModuleList()
        
        for _ in range(self.block_layer_nums - 1):  # -2 because we already added one layer and last layer is already defined
            self.layers.append(nn.Linear(self.hidden_dim,self.hidden_dim ))
        
        
        
        # Layer normalization
        self.layernorms = nn.ModuleList()
        for _ in range(self.block_layer_nums - 1):  # -1 because layer normalization is not applied to the last layer
            self.layernorms.append(nn.LayerNorm(self.hidden_dim))
        
    def forward(self, x):
        # Forward pass through the function f (MLP)
        out = x
        for i in range(self.block_layer_nums - 1):  # -1 because last layer is already applied outside the loop
            out = self.layers[i](out)
            out = self.layernorms[i](out)
            out = torch.relu(out)
        
        
        
        # Element-wise addition of input x and output of function f(x)
        out = x + out
        
        return out
    

class ResNN_pricing(nn.Module):
    def __init__(self,hyperparas):
        super().__init__()
        self.input_dim = hyperparas['input_dim']
        self.hidden_dim = hyperparas['hidden_dim']
        self.hidden_nums = hyperparas['hidden_nums']
        self.output_dim = hyperparas['output_dim']
        self.block_layer_nums = hyperparas['block_layer_nums']

        self.layer_list = []
        self.layer_list.append(nn.Sequential(nn.Linear(self.input_dim,self.hidden_dim),nn.ReLU() ) )

        for _ in range(self.hidden_nums-1):
            self.layer_list.append(ResNetBlock(hyperparas)
                                   )

        self.layer_list.append(nn.Linear(self.hidden_dim,self.output_dim))

        self.linear_stock = nn.Sequential(*self.layer_list)

    def forward(self,inputs):
        
        return self.linear_stock(inputs)

In [12]:
hyperparas = {'input_dim':5,'hidden_dim':64,'hidden_nums':10,'output_dim':88,'block_layer_nums':3}

res_pricing = ResNN_pricing(hyperparas).to(device=device,dtype=torch.float64)
loss_MSE = nn.MSELoss()
optim_Adam = torch.optim.Adam(res_pricing.parameters(),lr= 0.0001)


In [13]:
epochs =25
train_model(loss_MSE,optim_Adam,res_pricing,data_loader,test_data,epochs=epochs)

-----------------------Epoch: 0----------------------------------
Batch: 0,train loss is: 5.139003106586036
test loss is 4.520331972460173
Batch: 100,train loss is: 0.7633071433567397
test loss is 0.7723634582914265
Batch: 200,train loss is: 0.2868414586561597
test loss is 0.3146973447192724
Batch: 300,train loss is: 0.16219731549888558
test loss is 0.16251347347227765
-----------------------Epoch: 1----------------------------------
Batch: 0,train loss is: 0.15975359057811547
test loss is 0.14948438930873842
Batch: 100,train loss is: 0.1162937502593498
test loss is 0.10681298089026733
Batch: 200,train loss is: 0.07377534001917141
test loss is 0.08682997125976542
Batch: 300,train loss is: 0.07934477968063293
test loss is 0.07043182217307149
-----------------------Epoch: 2----------------------------------
Batch: 0,train loss is: 0.05751288308702213
test loss is 0.06816677209990359
Batch: 100,train loss is: 0.08614609907536822
test loss is 0.05727501317737035
Batch: 200,train loss is: 0

In [14]:
y_test_pred = yinversetransform(res_pricing(torch.from_numpy(x_test_transform).to(device=device)).cpu().detach().numpy())

div = y_test-y_test_pred

abs_div =np.abs(div)

print(f"mean volatility divergency is: {abs_div.mean()}")

mean volatility divergency is: 0.0017638782581835153


## Save models

In [15]:
torch.save(nn_pricing.state_dict(),'./YU/initialnn_Heston.pth')
torch.save(res_pricing.state_dict(),'./YU/res_layernorm_Heston.pth')